### For k merging

In [30]:
import torch, numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from datasets import load_dataset
from scipy.optimize import linear_sum_assignment

# -------------------------
# 1. Load model & tokenizer
# -------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2").to(device).eval()
model.config.pad_token_id = tokenizer.eos_token_id

# -------------------------
# 2. Extract FFN pre-activation features
# -------------------------
def extract_ffn_features(model, texts, max_tokens=4000):
    features = {i: [] for i in range(len(model.transformer.h))}
    hooks = []

    def create_hook(layer):
        def hook(module, inp, out):
            features[layer].append(out.detach().cpu())
        return hook

    for i, block in enumerate(model.transformer.h):
        hooks.append(block.mlp.c_fc.register_forward_hook(create_hook(i)))

    enc = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=64).to(device)
    with torch.no_grad():
        model(**enc)

    for h in hooks: h.remove()

    X = {}
    for i in features:
        x = torch.cat(features[i], dim=0)
        x = x.view(-1, x.shape[-1])
        if x.shape[0] > max_tokens:
            x = x[:max_tokens]
        X[i] = x.numpy()
    return X

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test[:1%]")
texts = [t for t in dataset['text'] if len(t.strip()) > 0][:64]
feature_mats = extract_ffn_features(model, texts)

# -------------------------
# 3. Get neuron alignment between layers using Hungarian algorithm
# -------------------------
def get_alignment(X_a, X_b):
    C = np.corrcoef(X_a, X_b, rowvar=False)
    C = C[:X_a.shape[1], X_a.shape[1]:]
    row_ind, col_ind = linear_sum_assignment(-C)
    return col_ind, float(C[row_ind, col_ind].mean())

# -------------------------
# 4. Permute FFN Weights for Alignment
# -------------------------
def permute_ffn(model, layer, perm):
    block = model.transformer.h[layer].mlp
    perm = torch.tensor(perm, dtype=torch.long, device=device)

    # GPT-2 weight shapes are transposed in your setup:
    # c_fc:   [768, 3072]
    # c_proj: [3072, 768]
    with torch.no_grad():
        block.c_fc.weight[:] = block.c_fc.weight[:, perm]
        block.c_fc.bias[:]   = block.c_fc.bias[perm]
        block.c_proj.weight[:] = block.c_proj.weight[perm, :]

# -------------------------
# 5. Merge k FFN layers: average aligned weights into the first layer
# -------------------------
def merge_k_layers(model, layers, perms):
    anchor = layers[0]
    blk_anchor = model.transformer.h[anchor].mlp

    W1 = blk_anchor.c_fc.weight.data.clone()
    b1 = blk_anchor.c_fc.bias.data.clone()
    W2 = blk_anchor.c_proj.weight.data.clone()
    b2 = blk_anchor.c_proj.bias.data.clone()

    for layer in layers[1:]:
        permute_ffn(model, layer, perms[layer])
        blk = model.transformer.h[layer].mlp
        W1 += blk.c_fc.weight
        b1 += blk.c_fc.bias
        W2 += blk.c_proj.weight
        b2 += blk.c_proj.bias

    k = len(layers)
    blk_anchor.c_fc.weight.data = W1 / k
    blk_anchor.c_fc.bias.data   = b1 / k
    blk_anchor.c_proj.weight.data = W2 / k
    blk_anchor.c_proj.bias.data   = b2 / k

# -------------------------
# 6. Compress Model by keeping only merged layers
# -------------------------
import copy

def compress_model(model, keep_layers, device="cuda" if torch.cuda.is_available() else "cpu"):
    # 1) build a full config dict from the original and change n_layer
    cfg_dict = model.config.to_dict()
    cfg_dict["n_layer"] = len(keep_layers)
    new_cfg = GPT2Config(**cfg_dict)

    # 2) instantiate compressed model
    compressed = GPT2LMHeadModel(new_cfg).to(device)

    with torch.no_grad():
        # 3) copy embeddings (handle positional length mismatch safely)
        compressed.transformer.wte.weight.copy_(model.transformer.wte.weight)
        # if compressed's wpe is smaller, slice original; if same, copy whole
        compressed.transformer.wpe.weight.copy_(
            model.transformer.wpe.weight[: compressed.transformer.wpe.weight.shape[0], :]
        )

        # 4) copy each kept layer's state_dict into the new model's layer
        for new_i, old_i in enumerate(keep_layers):
            compressed.transformer.h[new_i].load_state_dict(
                copy.deepcopy(model.transformer.h[old_i].state_dict())
            )

        # 5) copy final layernorm and lm_head
        compressed.transformer.ln_f.weight.copy_(model.transformer.ln_f.weight)
        compressed.transformer.ln_f.bias.copy_(model.transformer.ln_f.bias)
        compressed.lm_head.weight.copy_(model.lm_head.weight)

        # 6) ensure config tokens are consistent
        if hasattr(model.config, "pad_token_id"):
            compressed.config.pad_token_id = model.config.pad_token_id
        compressed.config.vocab_size = model.config.vocab_size

    return compressed


# -------------------------
# 7. RUN MERGING FOR ANY k
# -------------------------
k = 8  # change to 4,6,8,10 etc.

layer_groups = [list(range(i, min(i+k, len(model.transformer.h)))) for i in range(0, 12, k)]
perms = {}
# [[0,1,2,3], [4,5,6,7], [8,9,10,11]] for k=4
for group in layer_groups:
    anchor = group[0]
    for layer in group[1:]:
        perm, score = get_alignment(feature_mats[anchor], feature_mats[layer])
        perms[layer] = perm
        print(f"Aligned Layer {anchor} ↔ Layer {layer}, mean corr = {score:.4f}")
    merge_k_layers(model, group, perms)

# -------------------------
#  Save merged (full depth) model before compression
merged_dir = f"gpt2_merged_full_model{k}"
model.save_pretrained(merged_dir)
tokenizer.save_pretrained(merged_dir)
print(f" Saved merged full model to {merged_dir}")

compressed_model = compress_model(model, [g[0] for g in layer_groups])
# torch.save(compressed_model, f"compressed_model{k}.pth")
save_dir = f"gpt2_compressed_half_model{k}"   # save to a directory, not a file
compressed_model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)  # also save tokenizer for safety


Aligned Layer 0 ↔ Layer 1, mean corr = 0.3761
Aligned Layer 0 ↔ Layer 2, mean corr = 0.3878
Aligned Layer 0 ↔ Layer 3, mean corr = 0.3948
Aligned Layer 0 ↔ Layer 4, mean corr = 0.3776
Aligned Layer 0 ↔ Layer 5, mean corr = 0.3726
Aligned Layer 0 ↔ Layer 6, mean corr = 0.3724
Aligned Layer 0 ↔ Layer 7, mean corr = 0.3653
Aligned Layer 8 ↔ Layer 9, mean corr = 0.5122
Aligned Layer 8 ↔ Layer 10, mean corr = 0.4984
Aligned Layer 8 ↔ Layer 11, mean corr = 0.4677
 Saved merged full model to gpt2_merged_full_model8


('gpt2_compressed_half_model8/tokenizer_config.json',
 'gpt2_compressed_half_model8/special_tokens_map.json',
 'gpt2_compressed_half_model8/vocab.json',
 'gpt2_compressed_half_model8/merges.txt',
 'gpt2_compressed_half_model8/added_tokens.json')

In [31]:
# -------------------------
# 8. Compare parameter counts + test generation
# -------------------------
def count_params(m): return sum(p.numel() for p in m.parameters())

orig = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
print("\nOriginal Params:", count_params(orig))

prompt = "India will become global leader in AI because"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    sample_out = orig.generate(**inputs, max_length=40)
print("\nGenerated output:")
print(tokenizer.decode(sample_out[0], skip_special_tokens=True))

# Iterate over compressed models
for k in [4, 6, 8, 10]:
    print(f"\n{'='*20}  Testing compressed_model{k}  {'='*20}")
    try:
        merged_model = GPT2LMHeadModel.from_pretrained(f"gpt2_merged_full_model{k}").to(device)
    except Exception as e:
        print(f"❌ Failed to load gpt2_merged_full_model{k}:", e)
        continue
    try:
        compressed_model_test = GPT2LMHeadModel.from_pretrained(f"gpt2_compressed_half_model{k}").to(device)
    except Exception as e:
        print(f"❌ Failed to load compressed_model{k}.pth:", e)
        continue

    print("Merged Params:", count_params(merged_model))
    print("Compressed Params:", count_params(compressed_model_test))

    # Generate text
    with torch.no_grad():
        out = compressed_model_test.generate(**inputs, max_length=40)
    print("Generated output:")
    print(tokenizer.decode(out[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Original Params: 124439808

Generated output:
India will become global leader in AI because of its ability to solve problems in a way that is not just a matter of human ingenuity but also of the human spirit.

The AI revolution is a

====================  Testing compressed_model4  ====================
Merged Params: 124439808
Compressed Params: 60648960
Generated output:
India will become global leader in AI becausecsvcsvcsv databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases databases

====================  Testing compressed_model6  ====================
Merged Params: 124439808
Compressed Params: 53561088
Generated output:
India will become global leader in AI because Secondly Secondly Secondly Secondly Secondly Secondly Secondly Secondly Secondly Secondly S

### perplexity

In [32]:
%pip install transformers datasets accelerate evaluate


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
### perplexity with out training 

In [34]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
import math

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
texts = [t for t in dataset['text'] if len(t.strip()) > 0][:100]  # use 100 lines for speed


def compute_perplexity(model_or_name, texts, batch_size=4):
    """Compute perplexity for either a model name or a preloaded model object."""
    if isinstance(model_or_name, str):
        model = GPT2LMHeadModel.from_pretrained(model_or_name).to(device).eval()
    else:
        model = model_or_name.to(device).eval()

    total_loss, total_tokens = 0.0, 0

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(
            batch,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        with torch.no_grad():
            loss = model(**inputs, labels=inputs["input_ids"]).loss

        num_tokens = inputs["input_ids"].numel()
        total_loss += loss.item() * num_tokens
        total_tokens += num_tokens

    avg_loss = total_loss / total_tokens
    return math.exp(avg_loss)


# --- Original model ---
orig_ppl = compute_perplexity("gpt2", texts)
print(f"Original GPT-2 Perplexity: {orig_ppl:.2f}")

# --- Compressed models ---
for k in [4, 6, 8, 10]:
    print(f"\n=== Evaluating compressed_model{k}.pth ===")
    try:
        compressed_model_test = GPT2LMHeadModel.from_pretrained(f"gpt2_compressed_half_model{k}").to(device)
    except Exception as e:
        print(f"❌ Failed to load gpt2_compressed_half_model{k}:", e)
        continue
    try:
        merged_model = GPT2LMHeadModel.from_pretrained(f"gpt2_merged_full_model{k}").to(device)
    except Exception as e:
        print(f"❌ Failed to load gpt2_merged_full_model{k}:", e)
        continue

    compressed_ppl = compute_perplexity(compressed_model_test, texts)
    merged_ppl = compute_perplexity(merged_model, texts)
    print(f"Merged GPT-2 (k={k}) Perplexity: {merged_ppl:.2f}")
    print(f"Compressed GPT-2 (k={k}) Perplexity: {compressed_ppl:.2f}")


Original GPT-2 Perplexity: 851.83

=== Evaluating compressed_model4.pth ===
Merged GPT-2 (k=4) Perplexity: 11748.72
Compressed GPT-2 (k=4) Perplexity: 398065.80

=== Evaluating compressed_model6.pth ===
Merged GPT-2 (k=6) Perplexity: 8714.57
Compressed GPT-2 (k=6) Perplexity: 247155.87

=== Evaluating compressed_model8.pth ===
Merged GPT-2 (k=8) Perplexity: 27535.84
Compressed GPT-2 (k=8) Perplexity: 2133693.73

=== Evaluating compressed_model10.pth ===
Merged GPT-2 (k=10) Perplexity: 4069.90
Compressed GPT-2 (k=10) Perplexity: 99429.67


### perplexity with training 

In [38]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import math

device = "cuda" if torch.cuda.is_available() else "cpu"
# 1. Tokenizer
# --------------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# --------------------------
# 2. Load + preprocess WikiText-2 properly
# --------------------------
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

def tokenize_function(examples):
    out = tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=128
    )
    out["labels"] = out["input_ids"].copy()
    return out

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized.set_format(type="torch")
train_data = tokenized["train"]
test_data = tokenized["test"]

# --------------------------
# 3. Training Function
# --------------------------
def train_model(model, save_dir):
    # Make sure gradients are enabled and model is in training mode
    torch.set_grad_enabled(True)
    model.train()

    args = TrainingArguments(
        output_dir=save_dir,
        eval_strategy="no",
        per_device_train_batch_size=2,
        num_train_epochs=2,
        logging_steps=2000,
        learning_rate=5e-5,
        weight_decay=0.01,
        report_to="none",
        save_strategy="no",    # or "epoch" or "no"
        save_steps=20000,         # save every 50k steps
        save_total_limit=1,       # keep only the last checkpoint
    )

    trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_data,
    tokenizer=tokenizer
    )
    trainer.train()

    model.save_pretrained(save_dir)

# --------------------------
# 4. Load & Train All Models
# --------------------------
models_to_train = {}

# Original GPT-2
models_to_train["gpt2_orig"] = "gpt2"

# Merged models
for k in [4, 6, 8, 10]:
    models_to_train[f"merged_model_{k}"] = f"gpt2_merged_full_model{k}"

# Compressed models
for k in [4, 6, 8, 10]:
    models_to_train[f"compressed_model_{k}"] = f"gpt2_compressed_half_model{k}"

for name, pa.0th_ in models_to_train.items():
    print(f"\n=== Training {name} ===")
    
    # Load model from directory
    try:
        model = GPT2LMHeadModel.from_pretrained(path_)
    except Exception as e:
        print(f"❌ Failed to load {path_}:", e)
        continue

    # Fine-tune
    train_model(model, save_dir=f"{name}_finetuned")



=== Training gpt2_orig ===


/tmp/ipykernel_11205/3405286741.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Step,Training Loss
2000,1.368300
4000,1.334500
6000,1.269900
8000,1.304400
10000,1.274700
12000,1.228600
14000,1.310700
16000,1.245700
18000,1.263100
20000,1.130900


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.



=== Training merged_model_4 ===


...

Step,Training Loss
2000,2.276100
4000,2.098600
6000,1.964700
8000,1.985200
10000,1.911700
12000,1.843300
14000,1.948900
16000,1.841800
18000,1.857400
20000,1.752900



=== Training compressed_model_8 ===


Step,Training Loss
2000,2.284100
4000,2.096200
6000,1.962400
8000,1.982500
10000,1.908200
12000,1.838600
14000,1.945500
16000,1.839600
18000,1.854300
20000,1.750700



=== Training compressed_model_10 ===


Step,Training Loss
2000,2.327600
4000,2.119800
6000,1.980200
8000,1.999500
10000,1.922700
12000,1.852900
14000,1.958700
16000,1.851100
18000,1.866500
20000,1.768100


In [39]:
import torch
import math
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer again
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load WikiText-2 Raw
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
test_texts = dataset["test"]   # we use raw text here.


# ----------------------------------------------------------------------
# 1. Perplexity Function (correct!)
# ----------------------------------------------------------------------
def compute_perplexity(model, data, tokenizer, max_samples=200, batch_size=2):
    model.eval()
    model.to(device)
    
    # Filter non-empty strings
    texts = [t for t in data["text"] if isinstance(t, str) and len(t.strip()) > 0][:max_samples]

    total_loss = 0.0
    total_tokens = 0

    for i in tqdm(range(0, len(texts), batch_size), desc="Computing perplexity"):
        batch = texts[i:i + batch_size]
        enc = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

        with torch.no_grad():
            out = model(**enc, labels=enc["input_ids"])
            loss = out.loss

            # Weighted loss
            total_loss += loss.item() * enc["input_ids"].numel()
            total_tokens += enc["input_ids"].numel()

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return perplexity


In [40]:
# Dictionary of saved model directories
model_dirs = {
    "gpt2_orig": "gpt2_orig_finetuned",
    "merged_4": "merged_model_4_finetuned",
    "merged_6": "merged_model_6_finetuned",
    "merged_8": "merged_model_8_finetuned",
    "merged_10": "merged_model_10_finetuned",
    "compressed_4": "compressed_model_4_finetuned",
    "compressed_6": "compressed_model_6_finetuned",
    "compressed_8": "compressed_model_8_finetuned",
    "compressed_10": "compressed_model_10_finetuned",
}


In [41]:
results = {}

for name, path in model_dirs.items():
    print(f"\n=== Evaluating perplexity for: {name} ===")

    try:
        model = GPT2LMHeadModel.from_pretrained(path)
    except Exception as e:
        print(f"❌ Could not load {path}: {e}")
        continue

    ppl = compute_perplexity(model, dataset["test"], tokenizer)
    results[name] = ppl

    print(f"➡️ Perplexity of {name}: {ppl:.4f}")



=== Evaluating perplexity for: gpt2_orig ===


Computing perplexity: 100%|██████████| 100/100 [00:15<00:00,  6.66it/s]


➡️ Perplexity of gpt2_orig: 12.2247

=== Evaluating perplexity for: merged_4 ===


Computing perplexity: 100%|██████████| 100/100 [00:14<00:00,  6.75it/s]


➡️ Perplexity of merged_4: 13.3651

=== Evaluating perplexity for: merged_6 ===


...

➡️ Perplexity of compressed_6: 33.2190

=== Evaluating perplexity for: compressed_8 ===


Computing perplexity: 100%|██████████| 100/100 [00:06<00:00, 15.13it/s]


➡️ Perplexity of compressed_8: 32.0007

=== Evaluating perplexity for: compressed_10 ===


Computing perplexity: 100%|██████████| 100/100 [00:06<00:00, 15.28it/s]

➡️ Perplexity of compressed_10: 31.8289


In [42]:
print("\n\n========== FINAL PERPLEXITY RESULTS ==========")
for name, ppl in results.items():
    print(f"{name:20s} : {ppl:.4f}")




========== FINAL PERPLEXITY RESULTS ==========
gpt2_orig            : 12.2247
merged_4             : 13.3651
merged_6             : 13.1584
merged_8             : 13.1134
merged_10            : 13.1680
compressed_4         : 28.4862
compressed_6         : 33.2190
compressed_8         : 32.0007
compressed_10        : 31.8289


In [43]:
import time
import torch

def measure_latency(model, tokenizer, device="cuda", num_samples=20, max_length=64):
    model.eval()
    model.to(device)

    texts = ["The quick brown fox jumps over the lazy dog."] * num_samples

    # Warm-up rounds
    for _ in range(3):
        _ = model.generate(
            **tokenizer(texts[0], return_tensors="pt").to(device),
            max_length=max_length
        )

    if device == "cuda":
        torch.cuda.synchronize()

    start = time.time()
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt").to(device)
        with torch.no_grad():
            _ = model.generate(**inputs, max_length=max_length)

    if device == "cuda":
        torch.cuda.synchronize()

    total = time.time() - start
    return total / num_samples  # average per sample


In [44]:
model_dirs = {
    "gpt2_orig": "gpt2_orig_finetuned",
    "merged_4": "merged_model_4_finetuned",
    "merged_6": "merged_model_6_finetuned",
    "merged_8": "merged_model_8_finetuned",
    "merged_10": "merged_model_10_finetuned",
    "compressed_4": "compressed_model_4_finetuned",
    "compressed_6": "compressed_model_6_finetuned",
    "compressed_8": "compressed_model_8_finetuned",
    "compressed_10": "compressed_model_10_finetuned"
}


In [47]:
from transformers import GPT2LMHeadModel

latency_results = {}

for name, path in model_dirs.items():
    print(f"\n=== Measuring latency for: {name} ===")
    
    try:
        model = GPT2LMHeadModel.from_pretrained(path)
    except Exception as e:
        print(f"❌ Failed to load {path}: {e}")
        continue

    latency = measure_latency(model, tokenizer, device=device)
    latency_results[name] = latency

    print(f"➡️ {name} Latency: {latency*1000:.2f} ms/sample")
 


=== Measuring latency for: gpt2_orig ===
➡️ gpt2_orig Latency: 73.90 ms/sample

=== Measuring latency for: merged_4 ===
➡️ merged_4 Latency: 71.28 ms/sample

=== Measuring latency for: merged_6 ===
➡️ merged_6 Latency: 84.85 ms/sample

=== Measuring latency for: merged_8 ===
➡️ merged_8 Latency: 83.57 ms/sample

=== Measuring latency for: merged_10 ===
➡️ merged_10 Latency: 71.61 ms/sample

=== Measuring latency for: compressed_4 ===
➡️ compressed_4 Latency: 590.37 ms/sample

=== Measuring latency for: compressed_6 ===
➡️ compressed_6 Latency: 30.50 ms/sample

=== Measuring latency for: compressed_8 ===
➡️ compressed_8 Latency: 502.06 ms/sample

=== Measuring latency for: compressed_10 ===
➡️ compressed_10 Latency: 31.12 ms/sample


In [48]:
orig_lat = latency_results["gpt2_orig"]

print("\n=========== LATENCY COMPARISON ===========")
for name, lat in latency_results.items():
    speed = orig_lat / lat
    print(f"{name:20s} : {lat*1000:.2f} ms   | speedup = {speed:.2f}x")



=========== LATENCY COMPARISON ===========
gpt2_orig            : 73.90 ms   | speedup = 1.00x
merged_4             : 71.28 ms   | speedup = 1.04x
merged_6             : 84.85 ms   | speedup = 0.87x
merged_8             : 83.57 ms   | speedup = 0.88x
merged_10            : 71.61 ms   | speedup = 1.03x
compressed_4         : 590.37 ms   | speedup = 0.13x
compressed_6         : 30.50 ms   | speedup = 2.42x
compressed_8         : 502.06 ms   | speedup = 0.15x
compressed_10        : 31.12 ms   | speedup = 2.37x


### IMDB

In [1]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

# ==================================================
# 1. Load Tokenizer
# ==================================================
tok = GPT2Tokenizer.from_pretrained("gpt2")
tok.pad_token = tok.eos_token

# ==================================================
# 2. Load & Tokenize IMDB
# ==================================================
ds = load_dataset("imdb")

def tok_fn(e):
    return tok(e["text"], truncation=True, padding="max_length", max_length=128)

ds = ds.map(tok_fn, batched=True, remove_columns=["text"])
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

collator = DataCollatorWithPadding(tokenizer=tok)

# ==================================================
# 3. GPT-2 Sentiment Classification Wrapper
# ==================================================
class GPT2ForSentiment(nn.Module):
    def __init__(self, gpt_model):
        super().__init__()
        self.gpt2 = gpt_model
        self.classifier = nn.Linear(self.gpt2.config.n_embd, 2)

    def forward(self, input_ids, attention_mask=None, labels=None):
        out = self.gpt2(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        # Take last non-padding token
        lengths = attention_mask.sum(dim=1) - 1
        last_hidden = out.hidden_states[-1][torch.arange(input_ids.size(0)), lengths, :]
        logits = self.classifier(last_hidden)

        loss = None
        if labels is not None:
            loss = nn.functional.cross_entropy(logits, labels)
        return {"loss": loss, "logits": logits}

# ==================================================
# 4. Training Arguments
# ==================================================
args = TrainingArguments(
    output_dir="./imdb_results",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    learning_rate=5e-5,
    logging_steps=50,
    save_strategy="no",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# ==================================================
# 5. Training & Evaluation Function
# ==================================================
def train_and_eval(model, name):
    print(f"\n==============================")
    print(f"🔹 Training {name}")
    print(f"==============================")

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds["train"],
        eval_dataset=ds["test"],
        tokenizer=tok,
        data_collator=collator
    )

    trainer.train()

    # Accuracy Evaluation
    model.eval()
    correct, total = 0, 0
    dl = torch.utils.data.DataLoader(ds["test"], batch_size=16)

    for batch in dl:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            out = model(batch["input_ids"], batch["attention_mask"])
        preds = out["logits"].argmax(dim=-1)
        correct += (preds == batch["label"]).sum().item()
        total += batch["label"].size(0)

    acc = correct / total
    print(f"✅ {name} IMDb Accuracy: {acc*100:.2f}%")

    # Save model
    save_dir = f"{name}_imdb_finetuned"
    model.gpt2.save_pretrained(save_dir)
    print(f"💾 Saved to {save_dir}")

    return acc

# ==================================================
# 6. Model Paths (Same as your perplexity setup)
# ==================================================
model_paths = {
    "gpt2_orig": "gpt2",
    "merged_4": "gpt2_merged_full_model4",
    "merged_6": "gpt2_merged_full_model6",
    "merged_8": "gpt2_merged_full_model8",
    "merged_10": "gpt2_merged_full_model10",
    "compressed_4": "gpt2_compressed_half_model4",
    "compressed_6": "gpt2_compressed_half_model6",
    "compressed_8": "gpt2_compressed_half_model8",
    "compressed_10": "gpt2_compressed_half_model10"
}

# ==================================================
# 7. Run Training for All Models
# ==================================================
results = {}

for name, path in model_paths.items():
    print(f"\n=== Loading base model: {name} ===")
    try:
        base = GPT2LMHeadModel.from_pretrained(path)
    except Exception as e:
        print(f"❌ Failed to load {path}: {e}")
        continue

    model = GPT2ForSentiment(base.to(device))
    acc = train_and_eval(model, name)
    results[name] = acc

# ==================================================
# 8. Print Summary Table
# ==================================================
print("\n================ FINAL IMDb Accuracies ================")
for name, acc in results.items():
    print(f"{name:20s} : {acc*100:.2f}%")


/home/user/miniconda3/envs/myexamenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-14 22:24:42.898957: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-14 22:24:43.246139: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-14 22:24:44.572883: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slight


=== Loading base model: gpt2_orig ===

🔹 Training gpt2_orig


/tmp/ipykernel_9882/87598760.py:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,1.371100
100,0.799800
150,0.737700
200,0.722300
250,1.419700
300,1.261800
350,1.068400
400,1.213200
450,0.754300
500,1.248000


✅ gpt2_orig IMDb Accuracy: 87.45%
💾 Saved to gpt2_orig_imdb_finetuned

=== Loading base model: merged_4 ===

🔹 Training merged_4


...

✅ compressed_6 IMDb Accuracy: 83.34%
💾 Saved to compressed_6_imdb_finetuned

=== Loading base model: compressed_8 ===

🔹 Training compressed_8


Step,Training Loss
50,1.127700
100,0.767900
150,0.748400
200,0.713100
250,0.765100
300,0.716400
350,0.722900
400,0.811900
450,0.704900
500,0.768600


✅ compressed_8 IMDb Accuracy: 82.95%
💾 Saved to compressed_8_imdb_finetuned

=== Loading base model: compressed_10 ===

🔹 Training compressed_10


Step,Training Loss
50,1.409100
100,0.756300
150,0.816300
200,0.747500
250,0.681300
300,0.718100
350,0.697700
400,0.766600
450,0.709300
500,0.772900


✅ compressed_10 IMDb Accuracy: 83.14%
💾 Saved to compressed_10_imdb_finetuned

================ FINAL IMDb Accuracies ================
gpt2_orig            : 87.45%
merged_4             : 82.99%
merged_6             : 83.34%
merged_8             : 82.94%
merged_10            : 83.07%
compressed_4         : 83.46%
compressed_6         : 83.34%
compressed_8         : 82.95%
compressed_10        : 83.14%


In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import evaluate
import numpy as np
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------
# 1. Dataset: CNN/DailyMail (first 100 test samples)
# --------------------------
data = load_dataset("cnn_dailymail", "3.0.0", split="test[:100]")  

# --------------------------
# 2. ROUGE Metric
# --------------------------
rouge = evaluate.load("rouge")

# --------------------------
# 3. Tokenizer (same for all GPT-based models)
# --------------------------
tok = GPT2Tokenizer.from_pretrained("gpt2")
tok.pad_token = tok.eos_token

# --------------------------
# 4. Models to Evaluate
# --------------------------
models = {   
    "gpt2_orig": "gpt2",
    "merged_4": "gpt2_merged_full_model4",
    "merged_6": "gpt2_merged_full_model6",
    "merged_8": "gpt2_merged_full_model8",
    "merged_10": "gpt2_merged_full_model10",
    "compressed_4": "gpt2_compressed_half_model4",
    "compressed_6": "gpt2_compressed_half_model6",
    "compressed_8": "gpt2_compressed_half_model8",
    "compressed_10": "gpt2_compressed_half_model10"
}

# --------------------------
# 5. Summarization Function
# --------------------------
def generate_summary(model, text, max_new_tokens=60):
    inp = tok(
        text, return_tensors="pt",
        truncation=True, padding=True, max_length=128
    ).to(device)

    output = model.generate(
        **inp,
        max_new_tokens=max_new_tokens,
        num_beams=3,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    return tok.decode(output[0], skip_special_tokens=True)

# --------------------------
# 6. Evaluation Loop
# --------------------------
results = {}

for name, path in models.items():
    print(f"\n🔹 Evaluating ROUGE for: {name}")
    
    model = GPT2LMHeadModel.from_pretrained(path).to(device).eval()
    predictions, references = [], []

    for i in tqdm(range(len(data))):
        article = data[i]["article"]
        ref = data[i]["highlights"]
        pred = generate_summary(model, article)
        predictions.append(pred)
        references.append(ref)

    score = rouge.compute(predictions=predictions, references=references)
    results[name] = score
    print(f"✅ ROUGE for {name}:", score)

# --------------------------
# 7. Final Comparison
# --------------------------
print("\n📊 Final ROUGE-L Comparison:")
for model_name, score in results.items():
    print(model_name, "→ ROUGE-L:", score["rougeL"])


Generating test split: 100%|██████████| 11490/11490 [00:00<00:00, 100469.18 examples/s]



🔹 Evaluating ROUGE for: gpt2_orig


 82%|████████▏ | 82/100 [00:24<00:05,  3.32it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 84%|████████▍ | 84/100 [00:25<00:04,  3.99it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 100/100 [00:29<00:00,  3.41it/s]


✅ ROUGE for gpt2_orig: {'rouge1': np.float64(0.23686015230383223), 'rouge2': np.float64(0.10447509226414367), 'rougeL': np.float64(0.1703181031849859), 'rougeLsum': np.float64(0.19939702492949057)}

🔹 Evaluating ROUGE for: merged_4


100%|██████████| 100/100 [00:30<00:00,  3.31it/s]


...

✅ ROUGE for compressed_6: {'rouge1': np.float64(0.2010496471219669), 'rouge2': np.float64(0.08842919377043754), 'rougeL': np.float64(0.14086239841847736), 'rougeLsum': np.float64(0.16460996729544008)}

🔹 Evaluating ROUGE for: compressed_8


100%|██████████| 100/100 [00:14<00:00,  6.98it/s]


✅ ROUGE for compressed_8: {'rouge1': np.float64(0.19911675148005076), 'rouge2': np.float64(0.08762151187074499), 'rougeL': np.float64(0.13940229051094813), 'rougeLsum': np.float64(0.16288697892987475)}

🔹 Evaluating ROUGE for: compressed_10


100%|██████████| 100/100 [00:14<00:00,  6.98it/s]


✅ ROUGE for compressed_10: {'rouge1': np.float64(0.21008277531349), 'rouge2': np.float64(0.09152118560110611), 'rougeL': np.float64(0.1466985625357335), 'rougeLsum': np.float64(0.1712538564874653)}

📊 Final ROUGE-L Comparison:
gpt2_orig → ROUGE-L: 0.1703181031849859
merged_4 → ROUGE-L: 0.16237754719411127
merged_6 → ROUGE-L: 0.16029673426510804
merged_8 → ROUGE-L: 0.15954930806658818
merged_10 → ROUGE-L: 0.18603887143869474
compressed_4 → ROUGE-L: 0.13732425935189357
compressed_6 → ROUGE-L: 0.14086239841847736
compressed_8 → ROUGE-L: 0.13940229051094813
compressed_10 → ROUGE-L: 0.1466985625357335
